## Notebook for Splitting Zone Data
None - (0)

1,5,7,9,10,4 - PZ (1)

2,3 - TZ (2)

6,8 - CZ (3)

Directories with zonal data: '..\Documents\Independent Study - MRI\data\data\'

1. Read the last number before .mat
2. Divide the zonal .mat files into PZ, TZ, CZ
3. 

In [1]:
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import scipy.ndimage

%run -i save_load
%run -i LoadZoneData

dir_path = os.path.join('.','data', 'data','data')

save_files = ['patient_data_test','zone_data_test']
#data= load_data_and_save(dir_path, save_files)

In [4]:
zones = ['PZ','TZ','CZ']
"""
PZ_zones = [1, 4, 5, 7, 9, 10]
TZ_zones = [2, 3]
CZ_zones = [6, 8]
patient = data[0]
for slice_index in range(patient['ADC'].shape[-1]):
            
    # To trim the slices
            
    # Check if the slice contains the prostate
    if np.max(patient['mask'][:,:,slice_index]) > 0:
        binary_mask = patient['mask'][:,:,slice_index] == 255  # create a binary mask 
        plt.imshow(binary_mask)
        plt.show()
        # For each modality compute feature extraction algorithms
        trimmed_mask = trim_zeros_2D(patient['CDI'][:,:,slice_index] * binary_mask) # apply the mask to the slice and trim zeroscdi
        out,out1,out2,out3 = feat_extraction_1d(trimmed_mask)
        
    # crop out the PZ values
    for zone_index in range(10):
        trimmed_zone_map = trim_zeros_2D(patient['zone_map'][:,:,slice_index]*binary_mask)
        if zone_index+1 in trimmed_zone_map:
            #print(zone_index)
            bin_mask = trimmed_zone_map == zone_index + 1
            adc_crop = trim_zeros_2D(patient['ADC'][:,:,slice_index]*binary_mask)[bin_mask == 1]
            m_crop = out[bin_mask == 1]

"""
#pdata = organize_pdata(data)

"\nPZ_zones = [1, 4, 5, 7, 9, 10]\nTZ_zones = [2, 3]\nCZ_zones = [6, 8]\npatient = data[0]\nfor slice_index in range(patient['ADC'].shape[-1]):\n            \n    # To trim the slices\n            \n    # Check if the slice contains the prostate\n    if np.max(patient['mask'][:,:,slice_index]) > 0:\n        binary_mask = patient['mask'][:,:,slice_index] == 255  # create a binary mask \n        plt.imshow(binary_mask)\n        plt.show()\n        # For each modality compute feature extraction algorithms\n        trimmed_mask = trim_zeros_2D(patient['CDI'][:,:,slice_index] * binary_mask) # apply the mask to the slice and trim zeroscdi\n        out,out1,out2,out3 = feat_extraction_1d(trimmed_mask)\n        \n    # crop out the PZ values\n    for zone_index in range(10):\n        trimmed_zone_map = trim_zeros_2D(patient['zone_map'][:,:,slice_index]*binary_mask)\n        if zone_index+1 in trimmed_zone_map:\n            #print(zone_index)\n            bin_mask = trimmed_zone_map == zone_ind

#### Convert the MATLAB files into python dictionary

Two formats:
1. pdata -> patient - zones - maps
2. zdata -> zones - patients - maps

In [107]:
zdata = organize_zdata(pdata)

save_obj(pdata,save_files[0])
save_obj(zdata,save_files[1])

In [2]:
pdata = load_obj(save_files[0])
zdata = load_obj(save_files[1])

#### Split patient-wise Training/Validation/Testing Sets
Ensure that there is a balance of cancer/non-cancer samples in each zone set

K-fold cross-validation on the training sets

In [5]:
from sklearn.model_selection import train_test_split

patients = list(pdata.keys())
#train_patients, test_patients = train_test_split(patients, test_size=0.2)

# saved split data into textfile
train_file = open("train_patients.txt", "r")
train_patients = train_file.read().split('\n')
import scipy.io
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

%run -i save_load

dir_path = os.path.join('.','data', 'data','data')
#load_data_and_save(dir_path)
test_file = open("test_patients.txt", "r")
test_patients = test_file.read().split('\n')
del train_patients[-1]
del test_patients[-1]

# Check the percentage of cancer for each zone in train/test set
train_pca = {}
test_pca = {}
train_num_samples = {}
test_num_samples = {}
for zone in zones:
    train_pca[zone] = 0
    test_pca[zone] = 0
    train_num_samples[zone] = 0
    test_num_samples[zone] = 0
    for patient in train_patients:
        train_pca[zone] += sum(1 for x in zdata[zone][patient]['GS'] if x > 6) # sum all the samples with cancer
        train_num_samples[zone] += len(zdata[zone][patient]['GS'])
    for patient in test_patients:
        test_pca[zone] += sum(1 for x in zdata[zone][patient]['GS'] if x > 6) # sum all the samples with cancer
        test_num_samples[zone] += len(zdata[zone][patient]['GS'])
    
    print('Train',zone, train_pca[zone],train_num_samples[zone], np.divide(train_pca[zone],train_num_samples[zone])*100)
    print('Test',zone, test_pca[zone], test_num_samples[zone], np.divide(test_pca[zone],test_num_samples[zone])*100)


Train PZ 129 17589 0.7334129285348797
Test PZ 33 4455 0.7407407407407408
Train TZ 93 6390 1.455399061032864
Test TZ 24 1587 1.5122873345935728
Train CZ 0 6318 0.0
Test CZ 0 1581 0.0


## Generate the feature maps for each 

In [120]:
p = train_patients[0]
len(pdata[p][zone]['ADC']['mean'])
len(pdata[p][zone]['GS'][0::3])

28

In [6]:
# Only for PZ
%run -i save_load

X_train, Y_train = gen_X_matrix(train_patients, 'PZ')
X_test, Y_test = gen_X_matrix(test_patients, 'PZ')
mri_testdata = pd.DataFrame.from_dict(X_test)
mri_testdata.head()

header = list(mri_testdata)

In [35]:
mods = ['ADC','CDI','HBV']
X_test = {}
Y_test = []

for mod in mods:
    X_test[mod] = []

for p in test_patients:
    for i in range(0,len(pdata[p]['PZ']['ADC'])):
        if pdata[p]['PZ']['GS'][i] > 6:
            pCa = '1'
        else:
            pCa = '0'
        if len(pdata[p]['PZ']['ADC'][i]) == len(pdata[p]['PZ']['CDI'][i]) == len(pdata[p]['PZ']['HBV'][i]):
            for j in range(0, len(pdata[p]['PZ']['ADC'][i])):
                if len(pdata[p]['PZ']['ADC'][i][j]) == len(pdata[p]['PZ']['CDI'][i][j]) == len(pdata[p]['PZ']['HBV'][i][j]):
                    dat = [d for d in pdata[p]['PZ']['ADC'][i][j] if d != 0]
                    Y_test.extend(pCa*len(dat))
                    for mod in mods:
                        data = [d for d in pdata[p]['PZ'][mod][i][j] if d != 0]
                        X_test[mod].extend(data)
                        
                        
print(len(X_test['ADC']))                  
#mri_testdata = pd.DataFrame.from_dict(X_test)

41527


## Balance out the training set for (+) and (-) samples

Using python kit imblearn

Methods of sampling
* OverSampling
* UnderSampling
* SMOTE
* Tomek
* Cluster Centroids

In [10]:
#Using ADC
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from imblearn.under_sampling import RandomUnderSampler
#from imblearn.combine import SMOTETomek
#smt = SMOTETomek(ratio='auto')
rus = RandomUnderSampler(random_state=0)
#cc = ClusterCentroids(ratio={'0': 10})

groups = ['CDI_orig']
X_t = np.array([X_train[i] for i in groups]).T
X_tt = np.array([X_test[i] for i in groups]).T

X_rus, Y_rus = rus.fit_sample(X_t, Y_train)
#X_rus, Y_rus = smt.fit_sample(X_t, Y_train)
# Standardize values
sc = StandardScaler().fit(X_t)

stand_X = sc.transform(X_t)
stand_X_test = sc.transform(X_tt)
mri_data = pd.DataFrame.from_dict(stand_X)

# Train classifier

#svc_adc = SVC(kernel='linear',class_weight='balanced')
#svc_adc.fit(mri_data, Y_train)
#y_model = svc_adc.predict(stand_X_test)

sgd_clf = linear_model.SGDClassifier(alpha=0.01, max_iter=100,class_weight='balanced')
sgd_clf.fit(mri_data, Y_train)
y_model = sgd_clf.predict(stand_X_test)

# Test classifier predictions
#accuracy_score(Y_test,y_model)
mat = confusion_matrix(Y_test, y_model)
print(mat)
print(sgd_clf.score(stand_X_test,Y_test))


[[ 8603 79375]
 [  120   189]]
0.0995843102608538


In [12]:
print(X_rus)

[[    0.        ]
 [    0.        ]
 [    0.        ]
 ...
 [13350.99311424]
 [15259.25251187]
 [15004.15828612]]


In [120]:
groups = ['CDI','ADC']
X_t = np.array([X_train[i] for i in groups])
X_tt = np.array([X_test[i] for i in groups])

sc = StandardScaler().fit(X_t.T)

stand_X = sc.transform(X_t.T)
stand_X_test = sc.transform(X_tt.T)
mri_data = pd.DataFrame(stand_X)

svc_cdi_adc = SVC(kernel='linear',class_weight='balanced')

svc_cdi.fit(mri_data[1:90000], Y_train[1:90000])
y_model = svc_cdi.predict(stand_X_test)
mat = confusion_matrix(Y_test, y_model)
mat

KeyboardInterrupt: 

## Using Ensemble Classification methods to mitigate the imbalanced dataset

* Bagging

In [83]:
#Using Bagging with n=10,max_samples=0.5
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

groups = ['CDI','HBV']
X_t = np.array([X_train[i] for i in groups]).T
X_tt = np.array([X_test[i] for i in groups]).T

#X_rus, Y_rus = rus.fit_sample(X_t, Y_train)
#X_rus, Y_rus = smt.fit_sample(X_t, Y_train)
# Standardize values
sc = StandardScaler().fit(X_t)

stand_X = sc.transform(X_t)
stand_X_test = sc.transform(X_tt)
mri_data = pd.DataFrame.from_dict(stand_X)

# Train classifier

#svc = SVC(kernel='linear',class_weight='balanced')
#svc_adc.fit(mri_data, Y_rus)
#y_model = svc_adc.predict(stand_X_test)

sgd_clf = linear_model.SGDClassifier(alpha=0.01, max_iter=100)
scores = cross_val_score(sgd_clf, stand_X_test, Y_test, cv=5)
#sgd_bag = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
#     max_depth=1, random_state=0, loss='deviance')
#sgd_bag.fit(mri_data, Y_train)
#y_model = sgd_bag.predict(stand_X_test)

# Test classifier predictions
#accuracy_score(Y_test,y_model)
#mat = confusion_matrix(Y_test, y_model)
#mat


C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\Maria\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassif

array([0.99316612, 0.99316612, 0.99316612, 0.99316612, 0.99316562])